In [21]:
import pandas as pd
from datetime import datetime,timedelta
import numpy as np

In [4]:
flights_fe = pd.read_csv(r"../data/2_featue_engineered/2_percent.csv")

In [5]:
weather_kaggle = pd.read_csv(r"../data/WeatherEvents_Jan2016-Dec2020.csv")

In [6]:
weather_kaggle = weather_kaggle.astype({'StartTime(UTC)':'datetime64[ns]',
                                       'EndTime(UTC)':'datetime64[ns]'})

In [7]:
weather_kaggle = weather_kaggle[(weather_kaggle["StartTime(UTC)"]>datetime(year=2019,month=1,day=1))&
                                (weather_kaggle["EndTime(UTC)"]<datetime(year=2020,month=2,day=1))]

In [26]:
def get_weather(data,percentages_to_print=[],num_row_processed=[0]):
    if num_row_processed[0] in percentages_to_print:
        print(f"{datetime.now().time()} : {100*num_row_processed[0]/total_num}% rows processed")
    num_row_processed[0] = num_row_processed[0]+1
    
    arr_time = data.arr_date_time
    dep_time = data.dep_date_time
    origin_id = data.origin
    dest_id = data.dest
    
    weather_details = pd.Series(dtype='string')
    
    try:
        weather_details["arr_type"] ,weather_details["arr_severity"] = weather_kaggle[(weather_kaggle["StartTime(UTC)"]<arr_time)&
                  (weather_kaggle["AirportCode"]=="K"+dest_id)].sort_values(by='StartTime(UTC)',ascending=False).iloc[0][['Type','Severity']]
    except IndexError as e:
        weather_details["arr_type"] ,weather_details["arr_severity"] = np.nan,np.nan
        
    try:
         weather_details["dep_type"] ,weather_details["dep_severity"] = weather_kaggle[(weather_kaggle["StartTime(UTC)"]<dep_time)&
                  (weather_kaggle["AirportCode"]=="K"+origin_id)].sort_values(by='StartTime(UTC)',ascending=False).iloc[0][['Type','Severity']]
    except IndexError as e:
        weather_details["dep_type"] ,weather_details["dep_severity"] = np.nan, np.nan
    
    return weather_details

In [28]:
num_row_processed = [0]
total_num = flights_fe.shape[0]
percentages = np.round(np.linspace(0,total_num,11))
weather_details = flights_fe.apply(get_weather,axis=1,percentages_to_print=percentages,num_row_processed = num_row_processed)

21:57:38.522406 : 0.0% rows processed
22:46:28.649382 : 9.999800853673918% rows processed
23:39:27.108675 : 20.000265528434777% rows processed
00:43:43.489429 : 30.000066382108695% rows processed
01:28:24.694482 : 39.99986723578261% rows processed
02:13:55.221228 : 50.00033191054347% rows processed
02:59:27.778813 : 60.00013276421739% rows processed
03:44:57.409271 : 69.9999336178913% rows processed
04:30:29.914869 : 79.99973447156522% rows processed
05:15:56.847445 : 90.00019914632608% rows processed


In [29]:
flights_fe = pd.merge(left=flights_fe,right=weather_details,left_index=True,right_index=True)

In [30]:
flights_fe.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,Rank_des,arr_day_of_week,arr_hour,dep_month,dep_day_of_week,dep_hour,arr_type,arr_severity,dep_type,dep_severity
0,2019-01-02,DL,DL_CODESHARE,DL,5303,9E,N606LR,5303,10397,ATL,...,143.0,2,17,1,2,15,Rain,Light,Rain,Light
1,2019-03-26,DL,DL_CODESHARE,DL,5454,9E,N914XJ,5454,10397,ATL,...,143.0,1,12,3,1,10,Rain,Light,Rain,Moderate
2,2019-04-17,DL,DL_CODESHARE,DL,5443,9E,N901XJ,5443,10397,ATL,...,143.0,2,17,4,2,15,Rain,Light,Rain,Light
3,2019-05-12,DL,DL_CODESHARE,DL,5371,9E,N197PQ,5371,10397,ATL,...,143.0,6,12,5,6,10,Rain,Light,Rain,Light
4,2019-06-09,DL,DL_CODESHARE,DL,5197,9E,N916XJ,5197,10397,ATL,...,143.0,6,17,6,6,15,Fog,Moderate,Rain,Light


In [31]:
flights_fe.to_csv(r"../data/2_featue_engineered/2_percent.csv",index=False)